In [ ]:
import pandas as pd
df = pd.read_csv("final_data.csv", index_col=0)

df.head()
df.isnull().sum()

,title,location,description,requirements,telecommuting,has_company_logo,has_questions,fraudulent,location_has_non_latin,description_has_non_latin,requirements_has_non_latin,has_requirements,description_and_requirements,country_state,country,has_location,has_location_details
0,Marketing Intern,"US, NY, New York","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,0,0,0,0,True,"Food52, a fast-growing, James Beard Award-winn...","US, NY",US,True,False
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,0,0,1,1,True,Organised - Focused - Vibrant - Awesome!Do you...,NaN,NZ,True,False
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever","Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,0,0,0,1,True,"Our client, located in Houston, is actively se...","US, IA",US,True,False
3,Account Executive - Washington DC,"US, DC, Washington",THE COMPANY: ESRI Environmental Systems Resea...,"EDUCATION:Bachelors or Masters in GIS, busines...",0,1,0,0,0,1,1,True,THE COMPANY: ESRI Environmental Systems Resea...,"US, DC",US,True,False
4,Bill Review Manager,"US, FL, Fort Worth",JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,0,0,1,1,True,JOB TITLE: Itemization Review ManagerLOCATION:...,"US, FL",US,True,False


In [ ]:
#############################################
##### Feature selection and engineering #####
#############################################
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine relevant text columns (you can modify this based on which columns you want to use)
# df['text'] = df['title'].fillna('') + ' ' + df['description'].fillna('')


X_numeric_cols = ['telecommuting', 'has_company_logo', 'has_questions', 'location_has_non_latin',
'description_has_non_latin', 'requirements_has_non_latin', 'has_requirements',
'has_location', 'has_location_details']
X_text_cols = ['title', 'description_and_requirements']
# [[['country_state', 'country']]]
y_col = ['fraudulent']

df2 = df[X_numeric_cols+X_text_cols+y_col].dropna()



# df2[X_numeric_cols] = df2[X_numeric_cols].astype(float)
df2["combined_text"] = df2["title"] + " " + df2["description_and_requirements"]
X_text_cols = ["combined_text"]
y_col = 'fraudulent'
X = df2[X_numeric_cols+X_text_cols]
y = df2[y_col]

In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score


# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit TfidfVectorizer

preprocessor = ColumnTransformer(
    transformers=[
        ("text_vectorized", TfidfVectorizer(max_features=5000, stop_words='english'), "combined_text"),
        ("num_scaled", MinMaxScaler(), X_numeric_cols)
    ],
    remainder='drop'
)

# Train logistic regression model
model = Pipeline([
    ("preprocess", preprocessor),
    ("clf", LogisticRegression(random_state=42, max_iter=1000))
])

model.fit(X_train, y_train)

# Make predictions and print classification report
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("F2 Score:", fbeta_score(y_test, y_pred, beta=2, average='binary'))
clf = model.named_steps["clf"]
print(clf.coef_)

/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)


Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3410
           1       0.97      0.55      0.70       257

    accuracy                           0.97      3667
   macro avg       0.97      0.77      0.84      3667
weighted avg       0.97      0.97      0.96      3667

F2 Score: 0.6010230179028133
[[-0.0251709   1.06088599 -0.26574446 ...  0.25869464 -0.42745866
  -0.030642  ]]


In [ ]:
from sklearn.metrics import fbeta_score, make_scorer
fbeta = make_scorer(fbeta_score, beta=2)